#### DocumentAnalysisClient

#### https://learn.microsoft.com/en-gb/azure/ai-services/document-intelligence/?view=doc-intel-4.0.0
#### https://documentintelligence.ai.azure.com/studio/read


#### DocumentIntelligenceClient

#### https://learn.microsoft.com/en-us/python/api/azure-ai-documentintelligence/azure.ai.documentintelligence.documentintelligenceclient?view=azure-python-preview

In [4]:
#! python -m pip install  -U --ignore-installed  azure-ai-vision-imageanalysis

In [3]:
#! pip install  -U --ignore-installed python-dotenv

In [50]:
#! pip install  -U --ignore-installed  pdf2image

In [27]:
#! pip install  -U --ignore-installed  azure-ai-formrecognizer --user

In [19]:
import os
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.ai.documentintelligence.models import AnalyzeResult
import time
import datetime
from IPython import embed
from dotenv import load_dotenv
import json
env_path = os.path.join("keys", ".env")
load_dotenv(env_path)

True

In [20]:
# Set the values of your computer vision endpoint and computer vision key
# as environment variables:
try:
    endpoint = os.getenv("ENDPOINT")
    key = os.getenv("KEY")
except KeyError:
    print("Missing environment variable 'VISION_ENDPOINT' or 'VISION_KEY'")
    print("Set them before running this sample.")
    exit()


# helper functions

def get_words(page, line):
    result = []
    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)
    return result


def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (
            word.span.offset + word.span.length
        ) <= (span.offset + span.length):
            return True
    return False


def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])
    

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )




In [52]:
help(DocumentAnalysisClient)

Help on class DocumentAnalysisClient in module azure.ai.formrecognizer._document_analysis_client:

class DocumentAnalysisClient(azure.ai.formrecognizer._form_base_client.FormRecognizerClientBase)
 |  DocumentAnalysisClient(endpoint: str, credential: Union[azure.core.credentials.AzureKeyCredential, azure.core.credentials.TokenCredential], **kwargs: Any) -> None
 |  
 |  DocumentAnalysisClient analyzes information from documents and images, and classifies documents.
 |  It is the interface to use for analyzing with prebuilt models (receipts, business cards,
 |  invoices, identity documents, among others), analyzing layout from documents, analyzing general
 |  document types, and analyzing custom documents with built models (to see a full list of models
 |  supported by the service, see: https://aka.ms/azsdk/formrecognizer/models). It provides different
 |  methods based on inputs from a URL and inputs from a stream.
 |  
 |  .. note:: DocumentAnalysisClient should be used with API versio

In [58]:
image_file = "data/pdf_test.pdf"

In [59]:
document_analysis_client

In [60]:
with open(image_file, mode="rb") as image_data:

    poller = document_analysis_client.begin_analyze_document(
            model_id= "prebuilt-read", document= image_data)
    result = poller.result()

In [61]:
output= result.to_dict()

In [62]:
output.keys()

dict_keys(['api_version', 'model_id', 'content', 'languages', 'pages', 'paragraphs', 'tables', 'key_value_pairs', 'styles', 'documents'])

In [63]:
len(output['pages'])

2

In [64]:
output.get('documents')

[]

In [69]:
output.get('paragraphs')

[{'role': None,
  'content': 'THE PII PROBLEM: PRIVACY AND A NEW CONCEPT OF PERSONALLY IDENTIFIABLE INFORMATION',
  'bounding_regions': [{'page_number': 1,
    'polygon': [{'x': 1.8375, 'y': 0.6467},
     {'x': 6.6562, 'y': 0.6376},
     {'x': 6.6582, 'y': 1.6789},
     {'x': 1.8395, 'y': 1.688}]}],
  'spans': [{'offset': 0, 'length': 81}]},
 {'role': None,
  'content': 'PAUL M. SCHWARTZ+ & DANIEL J. SOLOVE#',
  'bounding_regions': [{'page_number': 1,
    'polygon': [{'x': 2.3447, 'y': 1.9718},
     {'x': 6.141, 'y': 1.9718},
     {'x': 6.141, 'y': 2.1532},
     {'x': 2.3447, 'y': 2.1532}]}],
  'spans': [{'offset': 82, 'length': 37}]},
 {'role': None,
  'content': 'Personally identifiable information (PII) is one of the most central concepts in information privacy regulation. The scope of privacy laws typically turns on whether PII is involved. The basic assumption behind the applicable laws is that if PII is not involved, then there can be no privacy harm. At the same time, there is n

In [65]:
output['pages'][0].keys()

dict_keys(['page_number', 'angle', 'width', 'height', 'unit', 'lines', 'words', 'selection_marks', 'spans', 'barcodes', 'formulas'])

In [66]:
output['pages'][0]

{'page_number': 1,
 'angle': 0.0047690728679299355,
 'width': 8.5,
 'height': 11.0,
 'unit': 'inch',
 'lines': [{'content': 'THE PII PROBLEM:',
   'polygon': [{'x': 2.9941, 'y': 0.6445},
    {'x': 5.5107, 'y': 0.6398},
    {'x': 5.5107, 'y': 0.8689},
    {'x': 2.9941, 'y': 0.8737}],
   'spans': [{'offset': 0, 'length': 16}]},
  {'content': 'PRIVACY AND A NEW CONCEPT OF',
   'polygon': [{'x': 1.8385, 'y': 0.9119},
    {'x': 6.6567, 'y': 0.9119},
    {'x': 6.6567, 'y': 1.1458},
    {'x': 1.8385, 'y': 1.1506}],
   'spans': [{'offset': 17, 'length': 28}]},
  {'content': 'PERSONALLY IDENTIFIABLE',
   'polygon': [{'x': 2.3065, 'y': 1.184},
    {'x': 6.1888, 'y': 1.184},
    {'x': 6.1888, 'y': 1.418},
    {'x': 2.3065, 'y': 1.4132}],
   'spans': [{'offset': 46, 'length': 23}]},
  {'content': 'INFORMATION',
   'polygon': [{'x': 3.252, 'y': 1.4657},
    {'x': 5.2528, 'y': 1.4657},
    {'x': 5.2576, 'y': 1.6806},
    {'x': 3.252, 'y': 1.6853}],
   'spans': [{'offset': 70, 'length': 11}]},
  {'co

In [67]:
output.get('content')

'THE PII PROBLEM: PRIVACY AND A NEW CONCEPT OF PERSONALLY IDENTIFIABLE INFORMATION\nPAUL M. SCHWARTZ+ & DANIEL J. SOLOVE#\nPersonally identifiable information (PII) is one of the most central concepts in information privacy regulation. The scope of privacy laws typically turns on whether PII is involved. The basic assumption behind the applicable laws is that if PII is not involved, then there can be no privacy harm. At the same time, there is no uniform definition of PII in information privacy law. Moreover, computer science has shown that in many circumstances non-PII can be linked to individuals, and that de-identified data can be re-identified. PII and non-PII are thus not immutable categories, and there is a risk that information deemed non-PII at one time can be transformed into PII at a later juncture. Due to the malleable nature of what consti- tutes PII, some commentators have even suggested that PII be abandoned as the mechanism by which to define the boundaries of privacy la

In [68]:
for page in output['pages']:
    for line in page['lines']:
        print(line['content'])

THE PII PROBLEM:
PRIVACY AND A NEW CONCEPT OF
PERSONALLY IDENTIFIABLE
INFORMATION
PAUL M. SCHWARTZ+ & DANIEL J. SOLOVE#
Personally identifiable information (PII) is one of the most central concepts in
information privacy regulation. The scope of privacy laws typically turns on
whether PII is involved. The basic assumption behind the applicable laws is that if
PII is not involved, then there can be no privacy harm. At the same time, there is no
uniform definition of PII in information privacy law. Moreover, computer science
has shown that in many circumstances non-PII can be linked to individuals, and
that de-identified data can be re-identified. PII and non-PII are thus not immutable
categories, and there is a risk that information deemed non-PII at one time can be
transformed into PII at a later juncture. Due to the malleable nature of what consti-
tutes PII, some commentators have even suggested that PII be abandoned as the
mechanism by which to define the boundaries of privacy law.
